# Steps for all of us

Choose a dataset that you want to use. 

You may do whatever steps you think necessary for building the best classifier.

Take the data you chose and do whatever massaging you think is necessary: standardizing, scaling, feature engineering/ transforming, feature selection, etc.  

Build a classifier however you see fit. You may want build one and tweak the paramters manually or use some sort of grid search to look through all possible parameters. 

Remember: The same model built on the massaged data may perform better than if the data was untouched. It may be more conveniant to chose a standared massaging pipeline and tweak a model to that data.

After you massage your data, follow these steps:

if you want to balance your target (which you should) follow along these lines:

### build your features data and target data

- X = df.drop(columns = "whatever your target name is")
- y = df"whatever your target name is"

### split the data into training and testing

- X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

### create the oversampled data to train on 

- oversampler = SMOTE(random_state = 2019)
- X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

### Put the oversampled data back into a dataframe

- X_train_oversampled = pd.DataFrame(X_train_oversampled, columns = X_train.columns)
- y_train_oversampled = pd.Series(y_train_oversampled)

### Build your classifier here. As an example:

- xgb_clf = xgb.XGBClassifier(max_depth=5, n_estimators=100, colsample_bytree=0.3, learning_rate=0.1, n_jobs=-1)

 
### Fit to the oversampled data; this will train the classifier on the oversampled data

- xgb_clf.fit(X_train_oversampled, y_train_oversampled)

### Use 5-fold cross validation to see how well the classfier you built is doing on test data. 
Some points: you have to substitute your classifer name in the cross_val_score function 

- kfold = KFold(n_splits=5, random_state=2019)
- results = cross_val_score(xgb_clf, X_test, y_test, cv=kfold, scoring = 'f1')


## It may be best to keep all of your models you built; have a log of them to see their scores and keep a record of your process of building your data. 



## Building the stacking classifier

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from Modules import *
sns.set()
%matplotlib inline
import imblearn


### read in the final sequential data

In [2]:
#read in the data; data formated weird with extra column, 
#use only data we need
data = pd.read_csv('Data/Final_trimmed_sequential_data.csv')
df = data.iloc[:, 1:]

df.head()

,AGE,Y,SEX_Female,SEX_Male,EDUCATION_Graduate School,EDUCATION_Other,EDUCATION_University,MARRIAGE_Married,MARRIAGE_Non-married,PERCENT_OF_LIMIT_BAL1,...,PAY_4_Other,PAY_4_more_than_two_month_late,PAY_4_on_time,PAY_4_one_month_late,PAY_5_Other,PAY_5_more_than_two_month_late,PAY_5_on_time,PAY_6_Other,PAY_6_more_than_two_month_late,PAY_6_on_time
0,24,1,1,0,0,0,1,1,0,0.195650,...,0,0,1,0,1,0,0,1,0,0
1,26,1,1,0,0,0,1,0,1,0.022350,...,1,0,0,0,1,0,0,0,1,0
2,34,0,1,0,0,0,1,0,1,0.308011,...,1,0,0,0,1,0,0,1,0,0
3,37,0,1,0,0,0,1,1,0,0.899800,...,1,0,0,0,1,0,0,1,0,0
4,57,0,0,1,0,0,1,1,0,0.132340,...,1,0,0,0,1,0,0,1,0,0


In [13]:
#split my data
X = df.drop(columns = 'Y')
y = df['Y']

#split into training and testing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
Train_data, test_data = train_test_split(df, test_size = 0.2, random_state = 2019)

target = 'Y'
predictors = [x for x in Train_data.columns if x not in [target] ]

#create my oversampled data to train on
from imblearn.over_sampling import SMOTE
oversampler = SMOTE(random_state = 2019)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(Train_data[predictors], Train_data[target])

#Put the oversampled data back into a dataframe
X_Train_SMOTE = pd.DataFrame(X_train_oversampled, columns = Train_data[predictors].columns)
y_Train_SMOTE = pd.Series(y_train_oversampled)

# build my classifier; will start with a voting classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('RF', rnd_clf), ('svc', svm_clf)], 
    voting = 'hard', n_jobs=-1
)

#Fit to the oversampled data; this will train the classifier on the oversampled data
voting_clf.fit(X_train_oversampled, y_train_oversampled)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)), ('RF', RandomFo...f', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=-1, voting='hard', weights=None)

In [14]:
#Use 5-fold cross validation to see how well the classfier you built is doing on test data.

kfold = KFold(n_splits=5, random_state=2019)
results = cross_val_score(voting_clf, test_data[predictors], test_data[target], cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")

5-fold cross-validation results: 0.4289658885295835


## Gold Standard:

### Best model so far is the xgboost4 model

In [15]:
#compare the xgboost models

from sklearn.externals import joblib 
loaded_model = joblib.load('Models/xgboost4.dat')

results = cross_val_score(loaded_model, test_data[predictors], test_data[target], cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")


5-fold cross-validation results: 0.5424582112704233
